지금까지는 input과 output이 1대1로 매핑되는 시나리오를 보았다.

이번 절에서는 모든 input 데이터를 vector로 매핑한 다음 output vector로 decoding하는 결과를 가져오는 아키텍처를 구축하는 방법을 살펴본다.

영어로 입력된 텍스트를 프랑스어로 번역하는 예제로 볼 것이다.

# Getting ready

기계번역을 수행하기 위해 정의하게 될 아키텍처는 다음과 같다.

- 입력 문장과 해당 프랑스어 번역을 사용할 수 있는 레이블이 지정된 데이터 세트 가져오기
- 영어와 프랑스어 텍스트 각각에 자주 쓰이는 단어를 토큰화하고 추출한다.
  - 빈번한 단어를 식별하기 위해, 우리는 각 단어의 빈도를 센다.
  - 모든 단어의 전체 누적 빈도의 상위 80%를 구성하는 단어는 자주 사용되는 단어로 간주한다.
- 자주 사용하는 단어에 포함되지 않은 단어들은 unk(unknown) 기호로 대체한다.
- 각 단어에 ID를 할당한다.
- 입력 텍스트의 벡터를 fetch하는 인코더 LSTM을 작성
- 인코딩된 벡터를 dense layer를 통해 전달하여 각 time step에서 디코딩된 텍스트의 확률을 추출할 수 있다.
- model fit를 통해 ouput에서의 loss를 최소화한다.

# How to do it...

입력 텍스트를 번역하는 데 도움이 되는 모델 아키텍처가 여러개 있을 수 있다.

우리는 그 중 몇가지를 살펴볼 것이다.

## Preprocessing the data

입력 및 출력 데이터를 모델에 전달하려면 데이터셋을 전처리 해야한다.

### 1. Import the relevant packages and dataset:

In [1]:
import pandas as pd
import numpy as np
import string

from string import digits

import matplotlib.pyplot as plt

import re

from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, LSTM, Dense

import numpy as np
%matplotlib inline

Using TensorFlow backend.


In [0]:
!wget https://www.dropbox.com/s/2vag8w6yov9c1qz/english%20to%20french.txt

In [50]:
!ls

 atis.dict.intent.csv   atis.test.query.csv     atis.train.slots.csv
 atis.dict.slots.csv    atis.test.slots.csv     atis.zip
 atis.dict.vocab.csv    atis.train.intent.csv  'english to french.txt'
 atis.test.intent.csv   atis.train.pkl	        sample_data
 atis.test.pkl	        atis.train.query.csv


In [44]:
lines= pd.read_table('english to french.txt', names=['eng', 'fr'])

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [3]:
len(lines)

149861

### 2. 데이터셋에 약 15만 개의 문장이 있는 것을 감안하여, 모델 구축하기 위한 첫 5만개 문장-번역 쌍만을 사용하자.

In [45]:
lines = lines[0:50000]

In [46]:
lines[:5]

,eng,fr
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !


### 3. 입력 및 출력 텍스트를 소문자로 변환하고 구두점을 제거한다.

In [47]:
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.fr=lines.fr.apply(lambda x: x.lower())

In [48]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fr=lines.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

### 4. 출력 문장에서 'start' 토큰과 'end' 토큰을 추가한다.(프랑스 문장)

인코더 디코더 아키텍처에 유용할 수 있도록 'start' 토큰과 'end' 토큰를 추가한다.

그 이유는 뒤 섹션에서 설명

In [49]:
lines.fr = lines.fr.apply(lambda x : 'start '+ x + ' end')

In [50]:
lines.head()

,eng,fr
0,go,start va end
1,run,start cours end
2,run,start courez end
3,wow,start ça alors end
4,fire,start au feu end


In [51]:
lines.shape

(50000, 2)

### frequent 단어를 식별한다.

모든 단어의 전체 빈도 80%를 구성하는 빈도를 가진 단어를 frequent 단어로 정의한다.

In [12]:
# fit a tokenizer
from keras.preprocessing.text import Tokenizer
import json
from collections import OrderedDict

def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [13]:
eng_tokenizer = create_tokenizer(lines.eng)

In [14]:
type(eng_tokenizer)

keras_preprocessing.text.Tokenizer

In [15]:
eng_tokenizer = create_tokenizer(lines.eng)

# json 형태로 변환후 불러오는 작업 : dict 객체로 변환
output_dict = json.loads(json.dumps(eng_tokenizer.word_counts))

df = pd.DataFrame([output_dict.keys(), output_dict.values()]).T

df.columns = ['word','count']

df = df.sort_values(by='count',ascending = False)

df['cum_count'] = df['count'].cumsum()
df['cum_perc'] = df['cum_count']/df['cum_count'].max()
final_eng_words = df[df['cum_perc'] < 0.8]['word'].values

In [21]:
df.head()

,word,count,cum_count,cum_perc
0,start,50000,50000,0.158865
2,end,50000,100000,0.31773
20,je,9886,109886,0.34914
66,pas,5758,115644,0.367435
75,de,4663,120307,0.382251


In [17]:
final_eng_words[:5]

array(['i', 'you', 'a', 'is', 'the'], dtype=object)

입력에서 누적적으로 전체 영어 단어의 80%를 구성하는 영어 단어의 수를 추출한다.

In [26]:
def extract_cumsum_80(lang_tokenizer):
    # json 형태로 변환후 불러오는 작업 : dict 객체로 변환
    output_dict = json.loads(json.dumps(lang_tokenizer.word_counts))

    df = pd.DataFrame([output_dict.keys(), output_dict.values()]).T

    df.columns = ['word','count']

    df = df.sort_values(by='count',ascending = False)

    df['cum_count'] = df['count'].cumsum()
    df['cum_perc'] = df['cum_count']/df['cum_count'].max()
    final_lang_words = df[df['cum_perc'] < 0.8]['word'].values    
    return final_lang_words

In [27]:
final_fr_words = extract_cumsum_80(fr_tokenizer)

array(['start', 'end', 'je', 'pas', 'de'], dtype=object)

In [19]:
final_fr_words[:5]

array(['start', 'end', 'je', 'pas', 'de'], dtype=object)

프랑스어 단어도 마찬가지로 추출한다.

In [22]:
print(len(final_eng_words),len(final_fr_words))

384 357


### 6. 빈도가 낮은 단어를 필터링한다.

만약 어떤 단어가 자주 쓰이는 단어들 사이에 없다면, 우리는 그것을 알 수 없는 단어로 대체한다.

In [42]:
def filter_eng_words(x):
    t = []
    x = x.split()
    for i in range(len(x)):
        if x[i] in final_eng_words:
            t.append(x[i])
        else:
            t.append('unk')
        
    x3 = ''
    for i in range(len(t)):
        x3 = x3 + t[i] + ' '
        
    return x3

In [43]:
def filter_fr_words(x):
    t = []
    x = x.split()
    for i in range(len(x)):
        if x[i] in final_fr_words:
            t.append(x[i])
        else:
            t.append('unk')
        
    x3 = ''
    for i in range(len(t)):
        x3 = x3 + t[i] + ' '  
        
    return x3

In [33]:
def filter_lang_words(x, final_lang_words):
    t = []
    x = x.split()
    for i in range(len(x)):
        if x[i] in final_lang_words:
            t.append(x[i])
    else:
        t.append('unk')
        
    x3 = ' '.join(t)    
    return x3

문장을 입력으로 하고, unique 단어를 추출하며, 만약 단어가 빈번한 영어 단어(final_eng_words)  사이에 존재하지 않으면 unk로 대체한다.

프랑스어도 마찬가지로 처리한다.

예를 들어, 빈번한 단어와 빈번하지 않는 단어를 가진 임의의 문장은 다음과 같이 출력된다.

In [28]:
filter_eng_words('he is extremely good')

'he is good unk '

In [32]:
filter_lang_words('he is extremely good')

'he is good unk'

In [52]:
lines_copy = lines

In [53]:
lines['fr'] = lines['fr'].apply(filter_fr_words)
lines['eng'] = lines['eng'].apply(filter_eng_words)

In [54]:
lines[:5]

,eng,fr
0,go,start va end
1,run,start unk end
2,run,start unk end
3,unk,start ça unk end
4,unk,start au unk end


위 처리를 함수로 작성하여 모든 영어와 프랑스어 문장에 대해서 적용한다.

### 7. 영어(입력) 문장과 프랑스어(출력) 문장에 걸쳐 각 단어에 ID를 할당한다.

#### 1) 데이터(영어와 프랑스어 문장)에 모든 unique 단어의 list를 저장한다.

In [55]:
all_eng_words = set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_french_words=set()
for fr in lines.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)

In [59]:
list(all_eng_words)[:5]

['was', 'she', 'them', 'bed', 'met']

In [60]:
list(all_french_words)[:5]

['amis', 'pris', 'largent', 'colère', 'en']

In [63]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
# del all_eng_words, all_french_words

In [64]:
input_words[:5]

['a', 'about', 'afraid', 'again', 'agree']

In [65]:
target_words[:5]

['a', 'acheté', 'ai', 'aider', 'aije']

In [71]:
set(all_french_words) - set(final_fr_words) 

{'unk'}

In [72]:
len(all_eng_words)

385

In [73]:
len(target_words)

358

#### 2) 입력 단어의 사전과 그에 대응하는 index를 생성한다. 

In [66]:
input_token_index = dict(
    [(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i+1) for i, word in enumerate(target_words)])

In [67]:
num_decoder_tokens

358

### 8. 모든 문장의 길이가 동일하도록 입력과 target 문장의 최대 길이를 추출한다.

In [68]:
lenght_list=[]
for l in lines.fr:
    lenght_list.append(len(l.split(' ')))
fr_max_length = np.max(lenght_list)

In [69]:
fr_max_length

17

In [70]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
eng_max_length = np.max(lenght_list)

In [71]:
eng_max_length

8

데이터셋에 대한 여러 아키텍처의 성능을 비교해보자.

## Traditional many to many architecture

이 아키텍처에서 우리는 각 입력 단어를 128 차원 벡터로 embed 할 것이다. 그 결과 출력 벡터의 shape은 (batch_size, 128, 17) 이다.

이 버전에서 입력데이터가 17개의 time step을 가지고 있고 출력 데이터 또한 17개의 time step을 가지는 시나리오를 테스트하기 위해서 이 작업을 수행한다.

*여기서 17은 전체 문장 중에 최대 길이를 나타낸다.

### 1. input과 output 데이터셋 생성

'decoder_input_data' 및 'decoder_target_data'가 있다

우선, target 문장 단어에 해당하는 단어 ID로 'decoder_input_data'를 생성한다.

'decoder_target_data'는 'start' 토큰 이후 모든 워드에 대해 one-hot-encode된 target 데이터 버전이다.

In [72]:
encoder_input_data = np.zeros((len(lines.eng), fr_max_length), dtype='float32')
decoder_input_data = np.zeros((len(lines.fr), fr_max_length), dtype='float32')
decoder_target_data = np.zeros((len(lines.fr), fr_max_length, num_decoder_tokens+1), dtype='float32')

In [73]:
encoder_input_data.shape

(50000, 17)

In [74]:
encoder_input_data[:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]], dtype=float32)

In [76]:
decoder_input_data.shape

(50000, 17)

In [77]:
decoder_target_data.shape

(50000, 17, 359)

In [82]:
decoder_target_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [83]:
decoder_target_data[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

> 이전 섹션의 7b 단계에서 만든 사전에는 index 0에 해당하는 단어가 없으므로 'num_decodder_tokens'에 +1을 추가하고 있다는 점에 유의하자.

In [85]:
for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.fr)):
    
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
        
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:          
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            if t == len(target_text.split()) - 1:
                decoder_target_data[i, t:, 89] = 1

In [89]:
target_token_index['end']

89

영어 또는 프랑스어로 된 문장을 영어와 프랑스어로 대응하는 단어 ID로 대체하기 위해 입력 텍스트와 대상 텍스트를 반복한다.

이 결과를 모델에 전달할 수 있도록 디코더에 있는 target 데이터를 한 번에 인코딩하고 있다. 

또한, 모든 문장의 길이가 현재 동일하다는 점을 감안하여, 다음과 같은 루프에 문장 길이가 초과 된 후 89번째 색인(89는 최종 색인에 속함)에서 대상 데이터의 값을 1로 대체하고 있다.

In [86]:
print(decoder_input_data.shape,encoder_input_data.shape,decoder_target_data.shape)

(50000, 17) (50000, 17) (50000, 17, 359)


In [88]:
encoder_input_data[:5]

array([[120.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [264.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [264.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [336.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [336.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.]], dtype=float32)

In [87]:
decoder_input_data[:5]

array([[284., 321.,  89.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [284., 320.,  89.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [284., 320.,  89.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [284., 352., 320.,  89.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [284.,  18., 320.,  89.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.]], dtype=float32)

In [99]:
decoder_target_data[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [100]:
for i in range(decoder_input_data.shape[0]):
    for j in range(decoder_input_data.shape[1]):
        if(decoder_input_data[i][j]==0):
            decoder_input_data[i][j] = 89

앞의 코드에서는 디코더 입력 데이터의 0 값을 89로 대체하고 있다(89는 엔딩 토큰이고 0은 우리가 만든 단어 인덱스와 관련된 단어가 없기 때문에).

In [84]:
target_token_index['end']

89

In [35]:
print(decoder_input_data.shape, encoder_input_data.shape, decoder_target_data.shape)

(50000, 17) (50000, 17) (50000, 17, 359)


In [37]:
decoder_input_data[0]

array([284., 321.,  89.,  89.,  89.,  89.,  89.,  89.,  89.,  89.,  89.,
        89.,  89.,  89.,  89.,  89.,  89.], dtype=float32)

In [38]:
np.argmax(decoder_target_data[0],axis=1)

array([321,  89,  89,  89,  89,  89,  89,  89,  89,  89,  89,  89,  89,
        89,  89,  89,  89])

In [39]:
decoder_input_data[1]

array([284., 320.,  89.,  89.,  89.,  89.,  89.,  89.,  89.,  89.,  89.,
        89.,  89.,  89.,  89.,  89.,  89.], dtype=float32)

In [40]:
np.argmax(decoder_target_data[1],axis=1)

array([320,  89,  89,  89,  89,  89,  89,  89,  89,  89,  89,  89,  89,
        89,  89,  89,  89])

In [101]:
decoder_target_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [102]:
decoder_target_data[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

### 2. Build and fit the model, as follows:



In [0]:
import os
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten
from keras.layers import Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.layers import Embedding
from keras.layers import LSTM, RepeatVector, TimeDistributed, Dense, Bidirectional

In [42]:
# define NMT model
model = Sequential()
model.add(Embedding(len(input_words)+1, 128, input_length=fr_max_length, mask_zero=True))
model.add((Bidirectional(LSTM(256, return_sequences = True))))
#model.add(RepeatVector(fr_max_length))
model.add((LSTM(256, return_sequences=True)))
model.add((Dense(len(target_token_index)+1, activation='softmax')))
model.summary()

W0718 07:37:53.316706 139670635399040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 07:37:53.338088 139670635399040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 07:37:53.342557 139670635399040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0718 07:37:53.894759 139670635399040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instruct

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 17, 128)           49408     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 17, 512)           788480    
_________________________________________________________________
lstm_2 (LSTM)                (None, 17, 256)           787456    
_________________________________________________________________
dense_1 (Dense)              (None, 17, 359)           92263     
Total params: 1,717,607
Trainable params: 1,717,607
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

W0718 07:38:05.047342 139670635399040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0718 07:38:05.083129 139670635399040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [44]:
history = model.fit(encoder_input_data, decoder_target_data,
          batch_size=128,
          epochs=5,
          validation_split=0.05)

W0718 07:38:13.234614 139670635399040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 47500 samples, validate on 2500 samples
Epoch 1/5
47500/47500 [==============================] - 312s 7ms/step - loss: 3.5412 - acc: 0.3202 - val_loss: 3.1503 - val_acc: 0.3796
Epoch 2/5
47500/47500 [==============================] - 310s 7ms/step - loss: 2.5286 - acc: 0.4577 - val_loss: 2.6579 - val_acc: 0.4320
Epoch 3/5
47500/47500 [==============================] - 311s 7ms/step - loss: 2.1258 - acc: 0.5048 - val_loss: 2.3858 - val_acc: 0.4555
Epoch 4/5
47500/47500 [==============================] - 309s 6ms/step - loss: 1.8828 - acc: 0.5395 - val_loss: 2.2154 - val_acc: 0.4800
Epoch 5/5
47500/47500 [==============================] - 306s 6ms/step - loss: 1.7114 - acc: 0.5647 - val_loss: 2.1236 - val_acc: 0.4874


In [0]:
history_dict

In [0]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
epochs = range(1, len(val_loss_values) + 1)
import matplotlib.pyplot as plt
%matplotlib inline 

plt.subplot(211)
plt.plot(epochs, history.history['loss'], 'ro', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Test loss')
plt.title('Training and test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid('off')
plt.show()

plt.subplot(212)
plt.plot(epochs, history.history['acc'], 'ro', label='Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label='Test accuracy')
plt.title('Training and test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()]) 
plt.legend()
plt.grid('off')
plt.show()

In [0]:
encoder_input_data[0]

모델의 정확도 숫자는 정확도 측정에서도 'end' 토큰을 계산하기 때문에 오해의 소지가 있다는 점에 유의하십시오.

3. Calculate the number of words that were correctly translated:

In [0]:
count = 0
correct_count = 0
pred = model2.predict(encoder_input_data[47500:])
for i in range(2500):
    t = np.argmax(pred[i], axis=1)
    act = np.argmax(decoder_target_data[47500], axis=1)
    correct_count += np.sum((act == t) & (act != 89))
    count += np.sum(act != 89)
correct_count/count

In [0]:
decoder_input_data[47500+i] != 89

In [0]:
t

In [0]:
decoder_input_data[47500+i]

테스트 데이터(유효성 분할이 5%이므로 총 데이터 집합의 마지막 5%이다)에 대해 예측하고 있다.

 전체 단어의 ~19%가 정확하게 번역되었음을 알 수 있다.

## Many to hidden to many architecture

이전 아키텍처의 단점 중 하나는 입력에 어떤 입력이 있는 곳에서 최대 8개의 time step가 있다는 것을 알면서도 입력의 time step 수를 인위적으로 17개로 늘려야 한다는 것이었다.

이 아키텍처에서는 입력의 마지막 단계에서 숨겨진 상태 값을 추출하는 모델을 구축해 봅시다. 

또한 은닉 상태 값을 17회 복제한다(출력에는 17개의 시간 단계가 있으므로). 

복제된 숨겨진 시간 단계를 Dense 계층을 통과하여 출력에 있을 수 있는 클래스를 최종적으로 추출한다. 논리를 다음과 같이 코드화하자.

### 1. 입력에 8 time step, 출력에 17 time step이 되도록 입력 및 출력 데이터셋 생성

이는 입력에 17개의 time step이 있었고 현재 버전에서 8개의 time step이 있었기 때문에 이전 반복과는 다르다.

In [0]:
encoder_input_data = np.zeros((len(lines.eng), eng_max_length), dtype='float32')
decoder_input_data = np.zeros((len(lines.fr), fr_max_length), dtype='float32')
decoder_target_data = np.zeros((len(lines.fr), fr_max_length, num_decoder_tokens+1), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.fr)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
        
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t>0:          
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1
            if t == len(target_text.split()) - 1:
                decoder_target_data[i, t:, 89] = 1

for i in range(decoder_input_data.shape[0]):
    for j in range(decoder_input_data.shape[1]):
        if(decoder_input_data[i][j]==0):
            decoder_input_data[i][j] = 89

### 2. Build the model. 

RepeatVector 계층은 양방향 계층의 출력을 17회 복제한다는 점에 유의하십시오.

In [0]:
model2 = Sequential()
model2.add(Embedding(len(input_words)+1, 128, input_length=eng_max_length, mask_zero=True))
model2.add((Bidirectional(LSTM(256))))
model2.add(RepeatVector(fr_max_length))
model2.add((LSTM(256, return_sequences=True)))
model2.add((Dense(len(target_token_index)+1, activation='softmax')))

### 3. Compile and fit the model:

In [0]:
model2.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])
history1 = model2.fit(encoder_input_data, decoder_target_data,
          batch_size=128,
          epochs=5,
          validation_split=0.05)

In [0]:
history_dict = history1.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
epochs = range(1, len(val_loss_values) + 1)
import matplotlib.pyplot as plt
%matplotlib inline 

plt.subplot(211)
plt.plot(epochs, history1.history['loss'], 'ro', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Test loss')
plt.title('Training and test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid('off')
plt.show()

plt.subplot(212)
plt.plot(epochs, history1.history['acc'], 'ro', label='Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label='Test accuracy')
plt.title('Training and test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()]) 
plt.legend()
plt.grid('off')
plt.show()

In [0]:
encoder_input_data.shape

### 4. Calculate the % of total words that are correctly translated:

In [0]:
count = 0
correct_count = 0
pred = model2.predict(encoder_input_data[47500:])
for i in range(2500):
  t = np.argmax(pred[i], axis=1)
  act = np.argmax(decoder_target_data[47500],axis=1)
  correct_count += np.sum((act==t) & (act!=89))
  count += np.sum(act!=89)
correct_count/count

앞의 결과는 정확도가 19%로 이전의 반복과 거의 대등하다.

모든 입력 시간 단계의 정보가 마지막 숨겨진 계층 값에만 저장되었을 때 상당한 양의 정보가 손실되는 경향이 있기 때문에, 선행은 기대된다.

또한, 우리는 어느 시간 단계에서 잊혀져야 할 것에 대한 상당한 양의 정보를 포함하는 세포 상태를 사용하지 않는다.

## Encoder decoder architecture for machine translation

이전 섹션에서 정의한 아키텍처에는 두 가지 잠재적인 논리적 향상 기능이 있다.

1.번역을 생성하는 동안 셀 상태에 존재하는 정보를 활용한다.

2. 앞서 번역한 단어를 다음 단어의 예측에 입력으로 활용

두 번째 기법은 **Teacher Forcing**(교사 위조)라고 불린다. 

본질적으로 이전 시간 단계의 실제 값을 입력으로 주면서 현재 시간 단계를 생성함으로써 네트워크를 보다 빠르고, 실질적으로 더 정확하게 튜닝하고 있다.

### Getting ready

기계 번역 시스템을 구축하기 위해 채택할 인코더 디코더 아키텍처는 다음과 같다.

- 두 개의 decoder 데이터셋
    - `encoder_input_data`와 결합된 **decoder_input_data**가 입력이고 **decoder_target_data**가 출력이다.

  - **decoder_input_data**는 `start` 단어로 시작된다.

- 우리가 decoder에서 첫번째 단어를 예측할 때, 우리는 단어들의 input set을 벡터로 변환하고, 그 다음 input으로서 `start`를 decoder 모델에 통과시킨다. 예상되는 output은 output에서 `start`뒤의 첫 번째 단어이다.

- 비슷한 방식으로 output의 실제 첫 번째 단어가 input으로 두 번째 단어를 예측한다.

- 두 번째 단어를 예측하면서 출력의 실제 첫 단어가 입력인 유사한 방식으로 진행한다.

- 이 전략을 바탕으로 모델의 정확성을 계산한다.

### How to do it...

이를 통해, 이전 섹션에서 준비한 입력 및 출력 데이터셋에 대한 모델을 구축하자(이전 섹션의 아키텍처와 1단계는 동일).

#### 1. Build the model, as follows:

In [0]:
# We shall convert each word into a 128 sized vector
embedding_size = 128

##### 1) Prepare the encoder model:

In [0]:
encoder_inputs = Input(shape=(eng_max_length,))
en_x=  Embedding(num_encoder_tokens+1, embedding_size)(encoder_inputs)
en_x = Dropout(0.1)(en_x)
encoder = LSTM(256, return_sequences=True, unroll=True)(en_x)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

인코더 네트워크의 중간 계층을 추출하고 있으며 여러 데이터셋을 입력(인코더 입력 데이터 및 디코더 입력 데이터)으로 전달하기 때문에 기능 API를 사용하고 있다는 점에 유의하십시오.

##### 2. Prepare the decoder model:

In [0]:
decoder_inputs = Input(shape=(fr_max_length,))

dex=  Embedding(num_decoder_tokens+1, embedding_size)
#dex = Dropout(0.1)(dex)

decoder= dex(decoder_inputs)
decoder = Dropout(0.1)(decoder)

decoder = LSTM(256, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])
print('decoder', decoder)

#### 2. Build the model, as follows:

In [0]:
model3 = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

#### 3. Fit the model, as shown in the following code:

In [0]:
history3 = model3.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=32,epochs=5,validation_split=0.05)

#### 4. Calculate the % of words that are accurately transcribed:

In [0]:
act = np.argmax(decoder_target_data, axis=2)

count = 0
correct_count = 0
pred = model3.predict([encoder_input_data[47500:],decoder_input_data[47500:]])

for i in range(2500):
    t = np.argmax(pred[i], axis=1)
    correct_count += np.sum((act[47500+i] == t) & (act[47500+i] != 0))
    count += np.sum(decoder_input_data[47500+i] != 0)
correct_count/count

이 시나리오에서 전체 단어의 44%를 정확하게 번역했다는 점에 유의하십시오.

그러나 실시간 시나리오에서는 이 기능에 액세스할 수 없으므로 테스트 데이터 세트의 정확도를 계산할 때 'decoder_input_data'를 사용해서는 안 된다는 점에 유의하십시오.

이것은 우리가 이전 시간 단계에서 예측된 단어를 현재 시간 단계의 디코더 입력 단어로 사용할 것을 요구한다.

'decoder_input_data'를 'decoder_input_data_preed'로 다시 초기화하겠다.

In [0]:
decoder_input_data_pred = np.zeros((len(lines.fr),fr_max_length),dtype='float32')
final_pred = []
for i in range(2500):
    word = 284
    for j in range(17):
        decoder_input_data_pred[(47500+i), j] = word
        pred = model3.predict([encoder_input_data[(47500+i)].reshape(1,8), decoder_input_data_pred[47500+i].reshape(1,17)])
        t = np.argmax(pred[0][j])
        word = t
        if word == 89:
            break
    final_pred.append(list(decoder_input_data_pred[47500+i]))

앞의 코드에서 index 284는 `start`단어에 해당한다.

우리는 decoder input의 첫 번째 단어로 `start` 단어를 전달하고 다음 time step에 가장 높은 확률를 가진 단어를 예측한다.

두 번째 단어를 예측하면, `decoder_input_word_preed`를 업데이트하고, 세 번째 단어를 예측하고, ... `stop` 단어가 등장할 때 까지 계속한다.

이제 예상 번역된 단어를 수정했으므로, 번역의 정확성을 계산해 봅시다.

In [0]:
final_pred2 = np.array(final_pred)
count = 0
correct_count = 0
for i in range(2500):
    correct_count += np.sum((decoder_input_data[47500+i]==final_pred2[i]) & (decoder_input_data[47500+i]!=89))
    count += np.sum(decoder_input_data[47500+i]!=89)
correct_count/count

앞의 결과는 이 방법을 통해 정확하게 번역되는 모든 단어의 46%를 가져온다.

이전의 방법에서 번역의 정확성이 상당히 향상되어 있는 반면, 우리는 여전히 소스 언어에서 출발하는 단어들은 처음부터, 즉 대상 언어에서도, 즉 정렬이라는 단어를 고려하지 않을 수 있다는 직관을 취하고 있지 않다. 

다음 절에서는 단어 정렬의 문제를 해결하는 것을 검토할 것이다.

## Encoder decoder architecture with attention for machine translation

앞의 절에서는, 목표의 이전 시간 단계의 실제 단어가 모델에 대한 입력으로 사용되는 교사 강제 기법을 가능하게 함으로써 번역의 정확성을 높일 수 있다는 것을 배웠다.

이 절에서는 이 아이디어를 더 확장하고 각 단계에서 인코더와 디코더 벡터가 얼마나 유사한지에 기초하여 입력 인코더에 가중치를 할당한다. 

이런 식으로, 우리는 특정 단어가 암호기의 시간 단계에 따라 암호기의 숨겨진 벡터에 더 높은 가중치를 가질 수 있도록 하고 있다.

### How to do it...

이것으로, 주의 메커니즘과 함께 인코더 디코더 아키텍처를 구축하는 방법을 살펴보자.



#### 1. Build the encoder, as shown in the following code:

In [0]:
encoder_inputs = Input(shape=(eng_max_length,))
en_x= Embedding(num_encoder_tokens+1, embedding_size)(encoder_inputs)
en_x = Dropout(0.1)(en_x)
encoder = LSTM(256, return_sequences=True, unroll=True)(en_x)
encoder_last = encoder[:,-1,:]

#### 2. Build the decoder, as follows:

In [0]:
decoder_inputs = Input(shape=(fr_max_length,))
dex= Embedding(num_decoder_tokens+1, embedding_size)
decoder= dex(decoder_inputs)
decoder = Dropout(0.1)(decoder)
decoder = LSTM(256, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

앞의 코드에서, 우리는 디코더 아키텍처를 완성하지 못했다.
우리는 디코더에서 숨겨진 레이어 값을 추출했을 뿐이다.

#### 3. Build the attention mechanism. 

주의 메커니즘은 각 단계에서 숨겨진 벡터와 디코더 숨겨진 벡터가 얼마나 유사한지에 기초한다. 

이 유사성(가능한 모든 입력 시간 단계에서 최대 1개까지 합한 가중치를 제공하기 위해 수행되는 소프트맥스)에 기초하여, 다음과 같이 인코더 벡터에 가중치를 할당한다.

활성화 및 밀도 레이어를 통해 인코더 디코더 벡터를 전달하여 벡터 간에 도트 제품(비유사한 점—비유사한 점의 척도)을 취하기 전에 추가적인 비선형성을 달성하십시오.

In [0]:
t = Dense(5000, activation='tanh')(decoder)
t2 = Dense(5000, activation='tanh')(encoder)
attention = dot([t, t2], axes=[2, 2])

입력 시간 단계에 부여할 가중치 식별:

In [0]:
attention = Dense(eng_max_length, activation='tanh')(attention)
attention = Activation('softmax')(attention)

가중 인코더 벡터를 다음과 같이 계산한다.

In [0]:
context = dot([attention, encoder], axes = [2,1])

#### 4. Combine the decoder and weighted encoder vector:

In [0]:
decoder_combined_context = concatenate([context, decoder])

#### 5. Connect the combination of decoder and weighted encoded vector to output layer:

In [0]:
output_dict_size = num_decoder_tokens + 1
decoder_combined_context = Dense(2000, activation='tanh')(decoder_combined_context)
output=(Dense(output_dict_size, activation="softmax"))(decoder_combined_context)

#### 6. Compile and fit the model, shown in the following code:

In [0]:
model4 = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[output])
model4.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])

In [0]:
model4.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=32, epochs=5, validation_split=0.05)

모형을 적합시키면 이 모델의 유효성 검사 손실이 이전 반복보다 약간 낫다는 것을 알게 될 것이다.

#### 7. Calculate the accuracy of translation in a similar way to what we did in the previous section:

In [0]:
decoder_input_data_pred=np.zeros((len(lines.fr), fr_max_length), dtype='float32')

final_pred_att = []
for i in range(2500):
  word = 284
  for j in range(17):
    decoder_input_data_pred[(47500+i), j] = word
    pred = model4.predict([encoder_input_data[(47500+i)].reshape(1,8),decoder_input_data_pred[47500+i].reshape(1,17)])
    t = np.argmax(pred[0][j])
    word = t
    if word==89:
      break
  final_pred_att.append(list(decoder_input_data_pred[47500+i]))      
  
final_pred2_att = np.array(final_pred_att)
count = 0
correct_count = 0
for i in range(2500):
  correct_count += np.sum((decoder_input_data[47500+i]==final_pred2_att[i]) & (decoder_input_data[47500+i]!=89))
  count += np.sum(decoder_input_data[47500+i]!=89)
correct_count/count  

앞의 코드는 정확히 번역된 전체 단어의 52%를 낳는데, 이는 이전의 반복보다 개선된 것이다.

#### 8. 합리적인 정확도를 가진 변환 시스템을 구축했으므로, 다음과 같이 테스트 데이터 세트의 몇 가지 변환(테스트 데이터 세트는 검증_분할을 5%로 지정한 총 데이터 세트의 마지막 5%임)을 검사해 봅시다.

In [0]:
k = -1500
t = model4.predict([encoder_input_data[k].reshape(1,encoder_input_data. shape[1]),decoder_input_data[k].reshape(1,decoder_input_data.shape[1])]).reshape(decoder_input_data.shape[1], num_decoder_tokens+1)

Extract the predicted translations in terms of words:

In [0]:
t2 = np.argmax(t,axis=1)
for i in range(len(t2)):
  if int(t2[i])!=0:
    print(list(target_token_index.keys())[int(t2[i]-1)])

> The output of the preceding code after converting English sentence to French is as follows:

Extract the actual translations in terms of words:

In [0]:
t2 = decoder_input_data[k]
for i in range(len(t2)):
  if int(t2[i])!=89:
    print(list(target_token_index.keys())[int(t2[i]-1)])

우리는 예측된 번역이 원래의 번역에 상당히 가깝다는 것을 안다. 

이와 유사한 방식으로 검증 데이터 세트에 대해 몇 가지 더 자세히 알아보도록 합시다.

앞에서 우리는 괜찮은 번역이 있다는 것을 알 수 있지만, 몇 가지 잠재적 개선 영역이 있다.

- 단어 유사성에 대한 설명:
  - 제나 제이와 같은 단어들은 상당히 비슷하기 때문에 정확도를 떨어뜨리고 있음에도 불구하고 심하게 처벌되어서는 안 된다
  
- unk 수 감소:
  - 데이터 세트의 차원을 줄이기 위해 Unk 단어 수를 줄임
  - 더 큰 코퍼스를 수집하고 산업 규모 구성을 갖춘 기계에서 작업할 때, 우리는 잠재적으로 높은 치수 데이터를 작업할 수 있다